# COVID-19 Analysis and Visualization

In [1]:
import json
import urllib
import calendar
import numpy as np
import pandas as pd
import folium
from folium.plugins import Search
import branca
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.style
import plotly.express as px
import plotly.graph_objs as go
from scipy.stats import pearsonr
from IPython.core.display import HTML

In [32]:
# options (date range, export figure)
end=9
export=True

In [33]:
period = list(range(3,end+1))
period_m=list()
for i in period:
    period_m.append(list(calendar.month_name)[i])

## Geospatial data

In [34]:
counties_gdf=gpd.read_file('../datasets/ca_counties_simplified.geojson').set_index('NAME').sort_index()
counties_gdf.head()

,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
NAME,,,,,
Alameda,1914242789,212979931,37.6471385,-121.9124880,"POLYGON ((-122.28179 37.70823, -122.10959 37.4..."
Alpine,1912292633,12557304,38.6217831,-119.7983522,"POLYGON ((-120.07240 38.70277, -120.07250 38.5..."
Amador,1539933576,29470568,38.4435501,-120.6538563,"POLYGON ((-121.02752 38.50830, -121.02710 38.3..."
Butte,4238423343,105325812,39.6659588,-121.6019188,"POLYGON ((-122.04648 39.79765, -122.02457 39.7..."
Calaveras,2641820834,43806026,38.1838996,-120.5614415,"POLYGON ((-120.07250 38.50987, -120.12520 38.4..."


## Employment data

In [35]:
# Employment data https://data.edd.ca.gov/Industry-Information-/Industry-Employment-in-California-Counties/nt76-4rha
# get latest employment data on file for each county
# updated May 9, 2021

# read in csv
employment_df=pd.read_csv('../datasets/industry_employment_in_california_counties.csv')
employment_df['Area Name']=employment_df['Area Name'].str.replace(' County','')
# filter main industry data
employment_df = employment_df.loc[(employment_df['Series Code']%100000 == 0) & (employment_df['Series Code']>=11000000) ,:].reset_index(drop=True)
# limit to 2020
employment_df=employment_df[(employment_df.Year == 2020)& (employment_df['Month'].isin(period_m))].reset_index(drop=True)

# make date column to sort dataframe chronologically
employment_df['Date'] = employment_df['Month'].astype(str) + employment_df['Year'].astype(str)
employment_df['Date'] = pd.to_datetime(employment_df['Date'], format='%B%Y')
employment_df.sort_values(['Date','Area Name'],ascending=[False,True], inplace=True)

# make a list of counties with July data to later use as index
recent_df=employment_df.drop_duplicates(subset=['Area Name'], keep='first').set_index('Area Name',drop=True).sort_index()
county_index=list(recent_df.index)

# employment by industry,county for each month
monthlyemployment_df=employment_df.pivot_table(
        values='Current Employment', 
        index=['Area Name','Month'], 
        columns='Industry Title', 
        aggfunc=sum).fillna(0)

monthlyemployment_df.tail()

Industry Title       Construction  Durable Goods  \
Area Name Month                                    
Ventura   July            16800.0        18100.0   
          June            17200.0        18200.0   
          March           16800.0        19300.0   
          May             16900.0        18000.0   
          September       16500.0        18000.0   

Industry Title       Educational and Health Services  Financial Activities  \
Area Name Month                                                              
Ventura   July                               47200.0               15600.0   
          June                               47000.0               15600.0   
          March                              51500.0               15900.0   
          May                                46000.0               15500.0   
          September                          48300.0               15600.0   

Industry Title       Government  Information  Leisure and Hospitality  \
Area Name Month                                                         
Ventura   July          40400.0       3600.0                  28900.0   
          June          43700.0       3600.0                  28600.0   
          March         48000.0       4900.0                  37400.0   
          May           44800.0       3600.0                  23300.0   
          September     43800.0       3500.0                  29100.0   

Industry Title       Manufacturing  Natural Resources, Mining and Constructi  \
Area Name Month                                                                
Ventura   July             25400.0                                   17700.0   
          June             25500.0                                   18100.0   
          March            26500.0                                   17800.0   
          May              24500.0                                   17800.0   
          September        25500.0                                   17400.0   

Industry Title       Nondurable Goods  Other Services  \
Area Name Month                                         
Ventura   July                 7300.0          8200.0   
          June                 7300.0          7900.0   
          March                7200.0          9900.0   
          May                  6500.0          7000.0   
          September            7500.0          7800.0   

Industry Title       Professional and Business Services  Retail Trade  \
Area Name Month                                                         
Ventura   July                                  41800.0       34100.0   
          June                                  41600.0       33000.0   
          March                                 44100.0       36900.0   
          May                                   41100.0       29500.0   
          September                             41800.0       34600.0   

Industry Title       Total Farm  Trade, Transportation and Utilities  \
Area Name Month                                                        
Ventura   July          24600.0                              52800.0   
          June          29100.0                              51500.0   
          March         25700.0                              56100.0   
          May           30000.0                              47800.0   
          September     24100.0                              53400.0   

Industry Title       Transportation, Warehousing and Utilitie  Wholesale Trade  
Area Name Month                                                                 
Ventura   July                                         6000.0          12700.0  
          June                                         6000.0          12500.0  
          March                                        6100.0          13100.0  
          May                                          5900.0          12400.0  
          September                                    6100.0          12700.0

## Unemployment data

In [36]:
# https://www.labormarketinfo.edd.ca.gov/data/monthly-data-release.html
unemployment_df = pd.read_csv('../datasets/unemployment_edd.csv')
# limit to 2020
unemployment_df = unemployment_df[(unemployment_df['areatype']==4) & (unemployment_df['periodyear']==2020) & (unemployment_df['Adjusted']=='Not adj')& (unemployment_df['period'].isin(period))]
unemployment_df['areaname'] = unemployment_df['areaname'].str.replace(' County','')
unemployment_df = unemployment_df.set_index('areaname',drop=True).sort_index().drop(columns=['stfips','areatype','area', 'periodyear', 'periodtype', 'Adjusted', 'Preliminary', 'benchmark'])
unemployment_gdf = counties_gdf.join(unemployment_df).drop(columns=['ALAND','AWATER','INTPTLAT','INTPTLON'])
unemployment_gdf.head()

,geometry,period,laborforce,emplab,unemp,unemprate
Alameda,"POLYGON ((-122.28179 37.70823, -122.10959 37.4...",3,836800,807500,29300,3.5
Alameda,"POLYGON ((-122.28179 37.70823, -122.10959 37.4...",4,799000,682000,117000,14.6
Alameda,"POLYGON ((-122.28179 37.70823, -122.10959 37.4...",5,800600,689800,110900,13.8
Alameda,"POLYGON ((-122.28179 37.70823, -122.10959 37.4...",6,813700,708200,105400,13.0
Alameda,"POLYGON ((-122.28179 37.70823, -122.10959 37.4...",7,811500,714400,97100,12.0


## COVID-19 data

In [37]:
# Cases https://data.chhs.ca.gov/dataset/covid-19-time-series-metrics-by-county-and-state

coviddata = pd.read_csv('../datasets/covid19cases_test.csv')
coviddata["month"] = pd.to_datetime(coviddata["date"]).dt.month
coviddata["year"] = pd.to_datetime(coviddata["date"]).dt.year
coviddata.rename(columns={'area':'county'},inplace=True)

# limit to 2020
coviddata=coviddata[(coviddata.year==2020) & (coviddata['month'].isin(period))]

## Population data

In [38]:
# California Census 2019 population estimates https://www.census.gov/content/census/en/search-results.html?stateGeo=none&q=california%20population%20ca&searchtype=web&page=1
pop_df = pd.read_csv('../datasets/population_estimates_by_county_2019.csv',header=1)
pop_df['County']=pop_df['County'].str.replace(' County','')
pop_df.head()

,County,Population
0,Alameda,"1,671,329"
1,Alpine,"1,129"
2,Amador,"39,752"
3,Butte,"219,186"
4,Calaveras,"45,905"


In [39]:
# create dataframe of new cases by month
monthlycases_df=coviddata.groupby(["county","month"]).sum()["cases"].unstack()
for column in monthlycases_df.columns:
    monthlycases_df.rename(columns={column:calendar.month_name[column]},inplace=True)

monthlycases_df=monthlycases_df.merge(pop_df, left_on='county', right_on='County').set_index('County',drop=True)
monthlycases_df.rename(columns={'Population':'pop'},inplace=True)
monthlycases_df['pop']=monthlycases_df['pop'].str.replace(',','').astype(int)

monthlycases_df.head()

,March,April,May,June,July,August,September,pop
County,,,,,,,,
Alameda,679.0,1265.0,1867.0,3474.0,6563.0,5404.0,2601.0,1671329
Alpine,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1129
Amador,5.0,3.0,2.0,24.0,155.0,86.0,25.0,39752
Butte,14.0,6.0,45.0,219.0,857.0,1235.0,552.0,219186
Calaveras,6.0,7.0,5.0,40.0,89.0,125.0,43.0,45905


In [40]:
# Plot Covid Cases per 100k by month
df = pd.DataFrame((monthlycases_df.iloc[:,:-1].div(monthlycases_df['pop'], axis=0)*100000).stack(level=0)).reset_index()
df.rename(columns={'level_1':'2020',0:'COVID-19 Cases per 100k'},inplace=True)
fig = px.line(df, x='2020', y='COVID-19 Cases per 100k', color="County",hover_name="County",title='COVID-19 Cases per 100k in California (CA)')
fig.update_layout(height=500,title_x=0.5)
fig.show()

if export:
    fig.write_html("../figures/1_covid_cases.html")
#fig.write_image("../figures/1_covid_cases.png")

Note: It appears some  the industry data we have underreports the size of the labor force, while other counties overreport it (relative to the labor force size in the unemployment data).

In [41]:
# Find distribution of COVID per major industry in each month (Mar 2020 to July 2020)
industryTotal = {}
industryTotalCase = {}
recentEmployment_df = pd.DataFrame() # only used to create percentages for industry
# Remove counties without industry data
coviddata = coviddata[coviddata['county'].isin(county_index)]
for month in range(3,period[-1]+1):
    # Pivot recent month employment data by (county vs industry)
    recentEmployment_df = pd.pivot_table(employment_df[(employment_df['Month']==calendar.month_name[month]) & (employment_df['Year']==2020)],index='Area Name',columns='Industry Title',values='Current Employment')
    # Merge with Unemployment data and sum
    recentEmployment_df = recentEmployment_df.merge(unemployment_df[unemployment_df['period']==month].loc[:,['unemp']],left_index=True,right_index=True)
    recentEmployment_df = recentEmployment_df.rename({'unemp': 'Unemployed'}, axis='columns')
    recentEmployment_df.loc['Total',:] = recentEmployment_df.sum(axis=0)    # Sum Industry
    recentEmployment_df['Total'] = recentEmployment_df.sum(axis=1)          # Sum County
    # Covert to percentage employment per industry
    percentEmployment_df = recentEmployment_df.div(recentEmployment_df['Total'],axis=0)
    # Total New cases per county in month
    recentCase = coviddata.groupby(["month","county"]).sum()["cases"].unstack().loc[month,:]
    # Distribution of cases per industry (Pseudo)
    percentCase_df = percentEmployment_df.mul(recentCase,axis=0)
    # Calc Percentage of cases per industry  
    percentCase_df.loc['Total Count',:] = percentCase_df.sum()
    percentCase_df.loc['Percent Count',:] = percentCase_df.loc['Total Count',:]/percentCase_df.loc['Total Count','Total']
    industryTotalCase[calendar.month_name[month]] = percentCase_df.transpose().loc[:,'Percent Count'].drop('Total')
    # Calculate industry distribution in month
    industryTotal[calendar.month_name[month]] = percentEmployment_df.transpose().loc[:,'Total'].drop('Total')
    

In [42]:
print(industryTotal.keys())
industryTotal[period_m[-1]].head()

dict_keys(['March', 'April', 'May', 'June', 'July', 'August', 'September'])


Construction                       0.030479
Durable Goods                      0.031229
Educational and Health Services    0.120781
Financial Activities               0.034101
Government                         0.106562
Name: Total, dtype: float64

In [43]:
print(industryTotalCase.keys())
industryTotalCase[period_m[-1]].head()

dict_keys(['March', 'April', 'May', 'June', 'July', 'August', 'September'])


Construction                       0.027837
Durable Goods                      0.028260
Educational and Health Services    0.121560
Financial Activities               0.031684
Government                         0.112155
Name: Percent Count, dtype: float64

In [44]:
#sort July industry totals, then sort all months by this
industryTotal[period_m[-1]]={k: v for k, v in sorted(industryTotal[period_m[-1]].items(), key=lambda item: item[1],reverse=True)}
for month in industryTotal:
    industryTotal[month] = pd.Series(dict(map(list, sorted(industryTotal[month].items(), key=lambda x:industryTotal[period_m[-1]][x[0]],reverse=True))))
for month in industryTotalCase:
    industryTotalCase[month] = pd.Series(dict(map(list, sorted(industryTotalCase[month].items(), key=lambda x:industryTotal[period_m[-1]][x[0]],reverse=True))))

In [45]:
# make dataframe for percentage differences to use in next plot
difference_df=((pd.DataFrame(industryTotalCase)-pd.DataFrame(industryTotal))*100).sort_index()
difference_df.head()

,March,April,May,June,July,August,September
Construction,-0.081164,-0.280010,-0.372042,-0.221552,-0.271698,-0.383712,-0.264197
Durable Goods,0.100338,-0.022554,-0.165978,-0.276951,-0.399075,-0.483035,-0.296893
Educational and Health Services,0.345686,0.623934,0.278663,-0.007244,-0.103509,-0.033823,0.077964
Financial Activities,0.078825,-0.002230,-0.127554,-0.182850,-0.313583,-0.382332,-0.241746
Government,-0.459719,-0.453405,0.240302,0.239584,0.338905,0.582537,0.559249


In [46]:
# plotly step update documentation https://chart-studio.plotly.com/~empet/15605/update-method-called-within-an-update/#/
df=pd.DataFrame(industryTotal)
df2=pd.DataFrame(industryTotalCase)
df3=difference_df
slider_data = []
colors = ['#296b9f',  '#f08536', '#ababab', '#595959','#6c9ecc','#4C4946','#004679','#296b9f',  '#f08536', '#ababab']
i=0
for month in df.columns:
    slider_data.append(go.Bar({
        'type':'bar',
        'marker':{'color':colors[i]},
        'x': list(df.index),
        'visible': True,
        'name': month,
        'y': df[month].values*100,
        'width':0.12}))
    i+=1
i=0
for month in df2.columns:
    slider_data.append(go.Bar({
        'type':'bar',
        'marker':{'color':colors[i]},
        'x': list(df2.index),
        'visible': False,
        'name': month,
        'y': df2[month].values*100,
        'width':0.12}))
    i+=1
i=0
for month in df3.columns:
    slider_data.append(go.Bar({
        'type':'bar',
        'marker':{'color':colors[i]},
        'x': list(df3.index),
        'visible': False,
        'name': month,
        'y': df3[month].values,
        'width':0.12}))
    i+=1
# Set initial slider/title index
start_index = 0


# Build slider steps
steps = [None,None,None]
steps[0] = dict(
    label='',
    method='update',
    args=[{'visible': list([True]*len(df.columns) + [False]*(len(df.columns))*2)},
           {'title.text': 'Proportion of Industries in CA',
             'yaxis.title':"Laborforce (%)",'yaxis.range':[0,16]}])

steps[1] = dict(
    label='',
    method='update',
    args=[{'visible': list([False]*len(df.columns) + [True]*len(df.columns) + [False]*len(df.columns))},
            {'title.text': 'Proportion of Industries in CA, Weighted by COVID-19 Cases',
             'yaxis.title':"Laborforce (%)",'yaxis.range':[0,16]}])

steps[2] = dict(
    label='',
    method='update',
    args=[{'visible': list([False]*(len(df.columns)*2) + [True]*len(df.columns))},
            {'title.text': 'Change in Proportion of Industries After Weighted by COVID-19 Cases',
             'yaxis.title':"Change (%)",'yaxis.range':[-2,2]}])

# Build sliders
sliders = [go.layout.Slider(
    active = 0,
    pad = {"t": -260,"l":50,"r":50},
    steps = steps,
    tickcolor='#FFFFFF'
)]

layout = go.Layout(
    sliders=sliders,
    title={'text':'Proportion of Industries in CA'},
    xaxis=dict(tickangle=270, tickfont=dict(size=10)),
    yaxis=dict(range=(0,16),title="Laborforce (%)"),
    height=540,title_x=0.5,
    bargap=0.2,
    bargroupgap=1,
    barmode='group',
    margin = dict(b = 100)
)

fig = go.Figure(data=slider_data, layout=layout)

fig.show()

if export:
    fig.write_html("../figures/2_proportion_industries_ca.html")
#fig.write_image("../figures/2_covid_total-farm-employment.png")

In [47]:
# sort df to use in next plot
difference_df['mean']=difference_df.mean(axis=1) # create mean to sort by
difference_df.sort_values('mean',inplace=True)
difference_df.drop(columns='mean',inplace=True)

In [48]:
# plotly step update documentation https://chart-studio.plotly.com/~empet/15605/update-method-called-within-an-update/#/
df=difference_df
data = []
colors = colors
i=0
for month in df.columns:
    data.append(go.Bar({
        'type':'bar',
        'marker':{'color':colors[i]},
        'x': list(df.index),
        'visible': True,
        'name': month,
        'y': df[month].values,
        'width':0.12}))
    i+=1

layout = go.Layout(
    title={'text':'Change in Proportion of Industries After Weighted by COVID-19 Cases - Sorted by Mean'},
    xaxis=dict(tickangle=270, tickfont=dict(size=10)),
    yaxis=dict(range=(-2,2),title="Change (%)"),
    height=540,title_x=0.5,
    bargap=0.2,
    bargroupgap=1,
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)

fig.show()

if export:
    fig.write_html("../figures/3_change_proportion_industries_ca.html")
#fig.write_image("../figures/3_change_proportion_industries_ca.png")


In [49]:
# calculate covid per 100k
unemployment_gdf['covid_per_100k']=pd.Series(dtype='float64')
unemployment_gdf['pop']=pd.Series(dtype='int64')
for month in period:
    unemployment_gdf.loc[unemployment_gdf['period']==month,'covid_per_100k']=((monthlycases_df[calendar.month_name[month]]/monthlycases_df['pop'])*100000)
    unemployment_gdf.loc[unemployment_gdf['period']==month,'pop']=monthlycases_df['pop']

### r values exploration

In [64]:
r_values_dict={}
r_values_dict['Unemployed']={}
for industry in monthlyemployment_df.columns:
    r_values_dict[industry]={}
    for month in industryTotal.keys():
        r_values_dict[industry][month]=(pearsonr(((pd.Series(monthlyemployment_df[monthlyemployment_df.index.get_level_values('Month') ==month].droplevel('Month')[industry])/(unemployment_gdf[(unemployment_gdf['period']==list(calendar.month_name).index(month))&(unemployment_gdf.index.isin(county_index))]['laborforce'])).round(2)*100), unemployment_gdf[(unemployment_gdf['period']==list(calendar.month_name).index(month))&(unemployment_gdf.index.isin(county_index))]['covid_per_100k'].fillna(0).values))[0]
for month in industryTotal.keys(): # add unemployed to table
    month_num=list(calendar.month_name).index(month)
    r_values_dict['Unemployed'][month]=(pearsonr(((pd.Series(unemployment_gdf[(unemployment_gdf['period'] ==month_num)]['unemprate'])*100)), unemployment_gdf[(unemployment_gdf['period']==month_num)]['covid_per_100k'].fillna(0).values))[0]


r_values_df=pd.DataFrame(r_values_dict).round(3)
#r_values_df[r_values_df.values<=-0.4]
#r_values_df[r_values_df.values>=.5]

r_values_df.sort_values(period_m[-1],axis=1,ascending=False,inplace=True)

r_values_df = r_values_df[['Total Farm','Unemployed','Leisure and Hospitality','Construction','Durable Goods',
                           'Educational and Health Services','Financial Activities','Government','Information',
                           'Manufacturing','Natural Resources, Mining and Constructi','Nondurable Goods',
                           'Other Services','Professional and Business Services','Retail Trade',
                           'Trade, Transportation and Utilities','Transportation, Warehousing and Utilitie','Wholesale Trade']]
r_values_df.rename(columns={'Leisure and Hospitality':'Leisure & Hospitality',
                           'Educational and Health Services':'Educational & Health Services',
                           'Government':'Govt.','Natural Resources, Mining and Constructi':'Natural Resources, Mining & Construction',
                           'Professional and Business Services':'Professional & Business Services',
                           'Trade, Transportation and Utilities':'Trade, Transp. & Utilities',
                            'Transportation, Warehousing and Utilitie':'Transp., Warehousing & Utilities'},inplace=True)

# EXPORT: split into two figures because of width
if export:
    r_values_df.iloc[:,:9].to_html('../figures/4_rvalues_1.html')
    r_values_df.iloc[:,9:].to_html('../figures/4_rvalues_2.html')
r_values_df

,Total Farm,Unemployed,Leisure & Hospitality,Construction,Durable Goods,Educational & Health Services,Financial Activities,Govt.,Information,Manufacturing,"Natural Resources, Mining & Construction",Nondurable Goods,Other Services,Professional & Business Services,Retail Trade,"Trade, Transp. & Utilities","Transp., Warehousing & Utilities",Wholesale Trade
March,-0.176,-0.309,0.633,0.057,0.067,-0.262,0.469,-0.112,0.185,-0.067,0.120,-0.039,0.043,0.258,0.084,-0.070,-0.112,0.064
April,0.201,0.257,0.162,0.105,0.171,0.127,0.254,-0.220,0.259,0.173,-0.001,0.112,-0.021,0.334,0.086,0.208,0.274,0.302
May,0.304,0.525,-0.150,-0.101,0.009,0.081,0.072,-0.017,0.033,0.022,-0.113,-0.018,-0.117,0.064,0.004,0.107,0.139,0.106
June,0.379,0.546,-0.226,0.007,0.001,0.141,0.098,-0.105,0.087,0.128,-0.052,0.097,-0.050,0.168,0.223,0.304,0.294,0.274
July,0.540,0.501,-0.185,-0.029,-0.013,0.080,0.079,-0.202,-0.030,0.319,-0.127,0.331,-0.035,0.118,0.167,0.342,0.438,0.395
August,0.556,0.332,-0.231,-0.078,-0.041,0.168,0.012,-0.296,-0.118,0.361,-0.157,0.278,0.019,0.042,0.202,0.347,0.422,0.313
September,0.511,0.303,-0.277,-0.133,0.054,0.314,0.056,-0.309,-0.051,0.294,-0.132,0.098,-0.032,0.064,0.215,0.324,0.343,0.329


In [51]:
for i in range(len(r_values_df.index)):
    print(calendar.month_name[i+3],list(r_values_df.iloc[i][(r_values_df.iloc[i].values>=0.5)].index[0:]),
        r_values_df.iloc[i][(r_values_df.iloc[i].values>=0.5)].values[0:])

March ['Leisure and Hospitality'] [0.633]
April [] []
May ['Unemployed'] [0.525]
June ['Unemployed'] [0.546]
July ['Total Farm', 'Unemployed'] [0.54  0.501]
August ['Total Farm'] [0.556]
September ['Total Farm'] [0.511]


### Scatter Plots

In [52]:
# make scatter
df = pd.DataFrame(
    {'Month':unemployment_gdf[(unemployment_gdf.index.isin(county_index))]['period'],
    'covid_per_100k':(unemployment_gdf[(unemployment_gdf.index.isin(county_index))]['covid_per_100k']),
    'pop':(unemployment_gdf[(unemployment_gdf.index.isin(county_index))]['pop'])}
)
df['Total Farm Employment Proportion']=pd.Series(dtype='float64')
for i in period:
    month=calendar.month_name[i]
    df['Total Farm Employment Proportion'][df['Month']==i]= (pd.Series(monthlyemployment_df[monthlyemployment_df.index.get_level_values('Month') ==month].droplevel('Month')['Total Farm'].fillna(0)/(unemployment_gdf[(unemployment_gdf.index.isin(county_index))]['laborforce'])).round(2)*100)
# Create a list of Plotly Go objects and r values (Pearsons) to dynamically update graph with slider
df.reset_index(inplace=True)
slider_data=[]
r_values=[]
for month in df['Month'].unique():
    if len(df[(df['Month']==month) & (df['covid_per_100k'].isnull())])>0: # remove null value for calculation (Modoc County has no covid reporting for march)
        null_index=int(df[(df['Month']==month) & (df['covid_per_100k'].isnull())].index.values[0])
        r_values.append((pearsonr(df[(df['Month']==month)]['Total Farm Employment Proportion'].values, df[(df['Month']==month)]['covid_per_100k'].fillna(0).values))[0])
    else:
        r_values.append((pearsonr(df[(df['Month']==month)]['Total Farm Employment Proportion'].values, df[(df['Month']==month)]['covid_per_100k'].values))[0])
    slider_data.append(go.Scatter({
        'mode':'markers',
        'marker': dict(color='#004679',size=(df[df['Month']==month]['pop'].values)** (1. / 4)/2+10,opacity=0.7,line=dict(width=1,color="#ffffff")),
        'name': '',
        'visible': False,
        'x': df[df['Month']==month]['Total Farm Employment Proportion'].values,
        'y': df[df['Month']==month]['covid_per_100k'].values,
        'customdata':df[(df['Month']==4)][["index","pop"]],
        'hovertemplate': (('<b>%{customdata[0]} County</b><br><br>'+\
                  '<i>Cases per 100k</i>: %{y}<br>'+\
                          '<i>Total Farm Laborforce</i>: %{x}%<br>'+\
                          '<i>Population</i>: %{customdata[1]}')) }))
    
# Set initial slider/title index
start_index = (period[-1]-3)

# Make initial trace visible
slider_data[start_index]['visible'] = True

# Build slider steps
steps = []
for i in range(len(slider_data)):
    step = dict(
        # Update method allows us to update both trace and layout properties
        method = 'update', 
        label = calendar.month_name[i+3],
        args = [
            # Make the ith trace visible
            {'visible': [t == i for t in range(len(slider_data))]},
            
            # Set the title for the ith trace
            {'title.text': 'COVID-19 Case Rate & Total Farm Laborforce of CA Counties, %s 2020 (R = %.3f)' % (calendar.month_name[i+3],r_values[i])}],
    )
    steps.append(step)

# Build sliders
sliders = [go.layout.Slider(
    active = (period[-1]-3),
    pad = {"t": 30},
    steps = steps
)]

layout = go.Layout(
    sliders=sliders,
    title={'text':'COVID-19 Case Rate & Total Farm Laborforce of CA Counties, %s 2020 (R = %.3f)' % (calendar.month_name[i+3],r_values[i])},
    xaxis=dict(range=(-0.5,30),title="Total Farm Laborforce (%)"),
    yaxis=dict(range=(-100,2000),title="COVID-19 Cases per 100k"),
    height=440,title_x=0.5,
)

fig = go.Figure(data=slider_data, layout=layout)

fig.show()

if export:
    fig.write_html("../figures/5a_covid_total-farm-employment.html")
#fig.write_image("../figures/5a_covid_total-farm-employment.png")

/Users/jacobwigal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jacobwigal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jacobwigal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jacobwigal/anaconda3/lib/python3.7/site-packages

In [53]:
# make scatter
df = unemployment_gdf.reset_index().rename(columns={'period':'Month'})

# Create a list of Plotly Go objects and r values (Pearsons) to dynamically update graph with slider
slider_data=[]
r_values=[]
for month in df['Month'].unique():
    if len(df[(df['Month']==month) & (df['covid_per_100k'].isnull())])>0: # remove null value for calculation (Modoc County has no covid reporting for march)
        null_index=int(df[(df['Month']==month) & (df['covid_per_100k'].isnull())].index.values[0])
        r_values.append((pearsonr(df[(df['Month']==month)]['unemprate'].values, df[(df['Month']==month)]['covid_per_100k'].fillna(0).values))[0])
    else:
        r_values.append((pearsonr(df[(df['Month']==month)]['unemprate'].values, df[(df['Month']==month)]['covid_per_100k'].values))[0])
    slider_data.append(go.Scatter({
        'mode':'markers',
        'marker': dict(color='#004679',size=(df[df['Month']==month]['pop'].values)** (1. / 4)/2+10,opacity=0.7,line=dict(width=1,color="#ffffff")),
        'name': '',
        'visible': False,
        'x': df[df['Month']==month]['unemprate'].values,
        'y': df[df['Month']==month]['covid_per_100k'].values,
        'customdata':df[(df['Month']==month)][["index","pop"]],
        'hovertemplate': (('<b>%{customdata[0]} County</b><br><br>'+\
                  '<i>Cases per 100k</i>: %{y}<br>'+\
                          '<i>Unemployment Rate</i>: %{x}%<br>'+\
                          '<i>Population</i>: %{customdata[1]}')) }))
    
# Set initial slider/title index
start_index = (period[-1]-3)

# Make initial trace visible
slider_data[start_index]['visible'] = True

# Build slider steps
steps = []
for i in range(len(slider_data)):
    step = dict(
        # Update method allows us to update both trace and layout properties
        method = 'update', 
        label = calendar.month_name[i+3],
        args = [
            # Make the ith trace visible
            {'visible': [t == i for t in range(len(slider_data))]},
            
            # Set the title for the ith trace
            {'title.text': 'COVID-19 Case Rate & Unemployment Rate of CA Counties, %s 2020 (R = %.3f)' % (calendar.month_name[i+3],r_values[i])}],
    )
    steps.append(step)

# Build sliders
sliders = [go.layout.Slider(
    active = (period[-1]-3),
    pad = {"t": 30},
    steps = steps
)]

layout = go.Layout(
    sliders=sliders,
    title={'text':'COVID-19 Case Rate & Unemployment Rate of CA Counties, %s 2020 (R = %.3f)' % (calendar.month_name[i+3],r_values[i])},
    xaxis=dict(range=(-0.5,30),title="Unemployment Rate (%)"),
    yaxis=dict(range=(-100,1350),title="COVID-19 Cases per 100k"),
    height=440,title_x=0.5,
)

fig = go.Figure(data=slider_data, layout=layout)

fig.show()

if export:
    fig.write_html("../figures/6a_covid_unemployment.html")
#fig.write_image("../figures/6a_covid_unemployment.png")

### Choropleth maps

In [54]:
# Cases and Farm Employment
from folium.plugins import DualMap
# left map
unemployment_gdf_sorted = unemployment_gdf[unemployment_gdf['period']==9].reset_index().sort_values(by='covid_per_100k', ascending=False)
unemployment_gdf_sorted.head(5).append(unemployment_gdf_sorted.tail(5))[['index','covid_per_100k']]
min, max = unemployment_gdf_sorted['covid_per_100k'].quantile([0.02,0.99]).apply(lambda x: round(x, 2))
mean = round(unemployment_gdf_sorted['covid_per_100k'].mean(),2)

colormap = branca.colormap.LinearColormap(
    colors=['#f0f3f7','#cad2e3','#BBC7DD','#839BBF','#49729E','#004679','#002c4d'],
    index=unemployment_gdf_sorted['covid_per_100k'].quantile([0,0.2,0.4,0.6,0.8,0.9,0.99]),
    vmin=min,
    vmax=max)

colormap.caption="COVID-19 Case Rate in CA, September 2020"

DualMap=DualMap(location=[37.5,-119], zoom_start=6)
ca_covid_map = DualMap.m1

style_function = lambda x: {
    'fillColor': colormap(x['properties']['covid_per_100k']),
    'color': 'black',
    'weight':1,
    'fillOpacity':0.8
}

stategeo = folium.GeoJson(
    unemployment_gdf_sorted,
    name='COVID-19 Case Rate',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['index', 'covid_per_100k'],
        aliases=['County', 'Cases per 100k'], 
        localize=True
    )
).add_to(ca_covid_map)

statesearch = Search(
    layer=stategeo,
    geom_type='Polygon',
    placeholder='Search for a County',
    collapsed=True,
    search_label='index',
    weight=3
).add_to(ca_covid_map)

colormap.add_to(ca_covid_map)

# right map

farm_gdf=unemployment_gdf_sorted.sort_index().set_index('index',drop=True)
farm_gdf['Total Farm Employment Proportion']=(pd.Series(monthlyemployment_df[monthlyemployment_df.index.get_level_values('Month') =='September'].droplevel('Month')['Total Farm']/(unemployment_gdf[(unemployment_gdf['period']==9) & 
        (unemployment_gdf.index.isin(county_index))]['laborforce'])).round(2)*100).reindex(counties_gdf.index, fill_value=-0.1)
farm_gdf=farm_gdf.rename(columns={0:'Total Farm Employment Proportion'}).sort_values(by='Total Farm Employment Proportion', ascending=False).reset_index()
farm_gdf.head(5).append(farm_gdf.tail(5))[['index','Total Farm Employment Proportion']]
min, max = farm_gdf['Total Farm Employment Proportion'].quantile([0.02,0.99]).apply(lambda x: round(x, 2))
mean = round(farm_gdf['Total Farm Employment Proportion'].mean(),2)

colormap2 = branca.colormap.LinearColormap(
    colors=['#feedde','#fdd0a2','#fdae6b','#fd8d3c','#f16913','#d94801','#8c2d04'],
    index=farm_gdf['Total Farm Employment Proportion'].quantile([0.25,0.3,0.4,0.6,0.8,0.9,0.99]),
    vmin=min,
    vmax=max
)

colormap2.caption="Farm Laborforce (%) in CA, September 2020"

ca_farm_map = DualMap.m2
map_dict = farm_gdf.set_index('index')['Total Farm Employment Proportion'].to_dict()

def get_color(x):
    value = map_dict.get(x['properties']['index'])
    if value <0:
        return '#bababa' # MISSING -> gray
    else:
        return colormap2(value)

style_function = lambda x: {
    'fillColor': get_color(x),
    'color': 'black',
    'weight':1,
    'fillOpacity':0.8
}
farm_gdf['Total Farm Employment Proportion'].replace(-0.1,'No Data Reported',inplace=True)
stategeo2 = folium.GeoJson(
    farm_gdf,
    name='Total Farm Employment Proportion',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['index', 'Total Farm Employment Proportion'],
        aliases=['County', 'Farm Laborforce (%)'], 
        localize=True
    )
).add_to(ca_farm_map)

colormap2.add_to(ca_farm_map)
output='../figures/5b_ca_covid_farm_map.html'
DualMap.save(output)

#fix legend
with open(output, 'r') as file:
    filedata=file.read()
filedata=filedata.replace('''.append("svg")
        .attr("id", 'legend')
        .attr("width", 450)''','''.append("svg")
        .attr("id", 'legend')
        .attr("width", 294)''')
filedata=filedata.replace('''.legend.leaflet-control").append("svg")
        .attr("id", 'legend')''','''.legend2.leaflet-control").append("svg")
        .attr("id", 'legend2')''',1)
filedata=filedata.replace('''legend = L.control({position: 'topright'});''','''legend2 = L.control({position: 'topright'});''',1)
filedata=filedata.replace('''.legend.onAdd = function (map) {var div = L.DomUtil.create('div', 'legend'); return div};''','''.legend2.onAdd = function (map) {var div = L.DomUtil.create('div', 'legend2'); return div};''',1)
filedata=filedata.replace('''legend.addTo''','''legend2.addTo''',1)
filedata=filedata.replace('''zoom: 6,''','''zoom: 5.5,
                    zoomDelta: 0.5,
                    zoomSnap: 0.25,
                    minZoom: 5.5,
                    maxZoom: 7,''')

with open(output, 'w') as file:
    file.write(filedata)
#DualMap

In [65]:
# Cases and Unemployment
from folium.plugins import DualMap
# left map
unemployment_gdf_sorted = unemployment_gdf[unemployment_gdf['period']==7].reset_index().sort_values(by='covid_per_100k', ascending=False)
unemployment_gdf_sorted.head(5).append(unemployment_gdf_sorted.tail(5))[['index','covid_per_100k']]
min, max = unemployment_gdf_sorted['covid_per_100k'].quantile([0.02,0.99]).apply(lambda x: round(x, 2))
mean = round(unemployment_gdf_sorted['covid_per_100k'].mean(),2)

colormap = branca.colormap.LinearColormap(
    colors=['#f0f3f7','#cad2e3','#BBC7DD','#839BBF','#49729E','#004679','#002c4d'],
    index=unemployment_gdf_sorted['covid_per_100k'].quantile([0,0.2,0.4,0.6,0.8,0.9,0.99]),
    vmin=min,
    vmax=max)

colormap.caption="COVID-19 Case Rate in CA, July 2020"

DualMap=DualMap(location=[37.5,-119], zoom_start=6)
ca_covid_map = DualMap.m1

style_function = lambda x: {
    'fillColor': colormap(x['properties']['covid_per_100k']),
    'color': 'black',
    'weight':1,
    'fillOpacity':0.8
}

stategeo = folium.GeoJson(
    unemployment_gdf_sorted,
    name='COVID-19 Case Rate',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['index', 'covid_per_100k'],
        aliases=['County', 'Cases per 100k'], 
        localize=True
    )
).add_to(ca_covid_map)

statesearch = Search(
    layer=stategeo,
    geom_type='Polygon',
    placeholder='Search for a County',
    collapsed=True,
    search_label='index',
    weight=3
).add_to(ca_covid_map)

colormap.add_to(ca_covid_map)

# right map
unemployment_gdf_sorted = unemployment_gdf[unemployment_gdf['period']==7].reset_index().sort_values(by='unemprate', ascending=False)
unemployment_gdf_sorted.head(5).append(unemployment_gdf_sorted.tail(5))[['index','unemprate']]
min, max = unemployment_gdf_sorted['unemprate'].quantile([0.02,0.99]).apply(lambda x: round(x, 2))
mean = round(unemployment_gdf_sorted['unemprate'].mean(),2)

colormap2 = branca.colormap.LinearColormap(
    colors=['#feedde','#fdd0a2','#fdae6b','#fd8d3c','#f16913','#d94801','#8c2d04'],
    index=unemployment_gdf_sorted['unemprate'].quantile([0,0.2,0.4,0.6,0.8,0.9,0.99]),
    vmin=min,
    vmax=max
)

colormap2.caption="Unemployment Rate (%) in CA, July 2020"

ca_unemployment_map = DualMap.m2

style_function = lambda x: {
    'fillColor': colormap2(x['properties']['unemprate']),
    'color': 'black',
    'weight':1,
    'fillOpacity':0.8
}

stategeo2 = folium.GeoJson(
    unemployment_gdf_sorted,
    name='Unemployment Rate',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['index', 'unemprate'],
        aliases=['County', 'Unemployment (%)'], 
        localize=True
    )
).add_to(ca_unemployment_map)

colormap2.add_to(ca_unemployment_map)
output='../figures/6b_ca_covid_unemployment_map.html'
DualMap.save(output)

#fix legend
with open(output, 'r') as file:
    filedata=file.read()
filedata=filedata.replace('''.append("svg")
        .attr("id", 'legend')
        .attr("width", 450)''','''.append("svg")
        .attr("id", 'legend')
        .attr("width", 294)''')
filedata=filedata.replace('''.legend.leaflet-control").append("svg")
        .attr("id", 'legend')''','''.legend2.leaflet-control").append("svg")
        .attr("id", 'legend2')''',1)
filedata=filedata.replace('''legend = L.control({position: 'topright'});''','''legend2 = L.control({position: 'topright'});''',1)
filedata=filedata.replace('''.legend.onAdd = function (map) {var div = L.DomUtil.create('div', 'legend'); return div};''','''.legend2.onAdd = function (map) {var div = L.DomUtil.create('div', 'legend2'); return div};''',1)
filedata=filedata.replace('''legend.addTo''','''legend2.addTo''',1)
filedata=filedata.replace('''zoom: 6,''','''zoom: 5.5,
                    zoomDelta: 0.5,
                    zoomSnap: 0.25,
                    minZoom: 5.5,
                    maxZoom: 7,''')

with open(output, 'w') as file:
    file.write(filedata)
#DualMap